# Pipeline validation helper

This notebook provides a synthetic label grid that can be fed to `classification.label_pieces` so the later stages of the pipeline can be exercised without running a classifier.

In [ ]:
from __future__ import annotations

import numpy as np

from pento import classification
from pento.segmentation import GRID_HEIGHT, GRID_WIDTH


## Synthetic label grid

The grid below encodes the 6×10 board with one pentomino label per cell. Replace this array with the output of your classifier when real predictions are available.

In [ ]:
synthetic_label_grid = np.array(
    [
        list('FFIILLPPNN'),
        list('FFIILLPPNN'),
        list('TTVVVWWWXX'),
        list('TTUUUVWWXX'),
        list('TUUUUVWWYY'),
        list('ZZZZZYYYYY'),
    ]
)

assert synthetic_label_grid.shape == (GRID_HEIGHT, GRID_WIDTH)
synthetic_label_grid


## Classification demo

`classification.label_pieces` accepts segmented cells. The helper below wraps the synthetic label grid in singleton height/width dimensions so it mimics the classifier output.

In [ ]:
grid_cells = synthetic_label_grid[..., None, None]
pieces = classification.label_pieces(grid_cells)

[(piece.name, int(piece.mask.sum())) for piece in pieces]
